Nezbytné importy.

Funkce `kdf (password, salt = b'', rounds = 1)` bude sloužit k zahashování hesla, jenž se bude předávat prvním parametrem `password`. Volitelně lze ještě zadat sůl a počet iterací.

In [2]:
import time
import hashlib
import sys
import itertools

# Výpočet hash z hesla s použitím soli a zvoleného počtu iterací
def kdf (password: bytes, salt: bytes = b'', rounds: int = 1) -> bytes:
  h = salt + password
  for i in range(rounds):
    dgst = hashlib.sha512()
    dgst.update(h)
    h = dgst.digest()
  return h # Výsledná hash

In [5]:
# Vyzkoušíme funkci kdf pro výpočet hashe hesla
print(kdf(b'AAA')) # bytes
print(kdf(b'AAA').hex()) # hex string

b'\x8dp\x8d\x18\xb5M\xf3\x96-io\x06\x9a\xd4-\xadwb\xb5\xd4\xd3\xc9~\xe5\xfa-\xae\x06s\xedFTQd\xc0x\xb8\xdb=Y\xc4\xb9` \xe41o\x17\xbb=\x91\xbf\x1fk\xc0\x89k\xbeuAn\xb8\xc3\x85'
8d708d18b54df3962d696f069ad42dad7762b5d4d3c97ee5fa2dae0673ed46545164c078b8db3d59c4b96020e4316f17bb3d91bf1f6bc0896bbe75416eb8c385


V souboru `English.dic` je seznam slov, která nám budou sloužit pro slovníkový útok, tzn. budeme uvažovat hesla pouze z tohoto seznamu. Pro následné rychlé získání hesla z hashe si předpočítáme hashe pro všechna slova v tomto seznamu.



In [6]:
# Načtení slov ze souboru
with open("English.dic", "r") as fin:
  passwords=fin.readlines()

passwords = [x.strip() for x in passwords] # Odstranění newline

In [7]:
# Vytvoříme slovník (dictionary - dále v textu budeme používat také "dict" pro rozlišení datového typu v Pythonu)
# - budou zde uloženy dvojice klíč:hodnota (hash:heslo v našem případě), indexuje se pomocí klíče
hashPasswordTable = {kdf(bytes(x, 'utf8')):x for x in passwords}

In [8]:
# Příklad použití slovníku: Zachytili jsme následující hash, a víme, že heslo je ze slovníku.
testHash = bytes.fromhex("8d708d18b54df3962d696f069ad42dad7762b5d4d3c97ee5fa2dae0673ed46545164c078b8db3d59c4b96020e4316f17bb3d91bf1f6bc0896bbe75416eb8c385")
# Jaké bylo odpovídající původní heslo?
print(hashPasswordTable[testHash])

AAA


## Úkol 1
Jak bude velká tato tabulka? Použitá hashovací funkce je SHA512.
Stačí řádově, neuvažujte reprezentaci slovníku (datového typu dict) v Pythonu.

**Odpověď:**
SHA512 hash funkce produkuje hash o délce 512 bitů, což je 64 bytů pro každé heslo.
Celková velikost hash tabulky v bytech: 318,971(English.dict) × 64 = 20,414,144 bytů, což je přibližně 19.5 MB.

Nyní můžeme pro srovnání zkusit rychlost nalezení hesla při použití hrubé síly (zkoušení všech možných kombinací povolených znaků hesla), nebo předpočítané tabulky.

**Poznámka:** Je dobré si uvědomit, že předpočítaná tabulka je pouze ze slov ve slovníku English.dic, tzn. je už sama o sobě značně omezená. Pokud bychom předpočítávali hash pro všechna možná hesla do určité délky, tabulka by byla značně větší.

In [ ]:
target = kdf(b'phial')

In [ ]:
%%time
for x in itertools.product('abcdefghijklmnopqrstuvwxyz', repeat=5): # procházení všech kombinací malých písmen o délce 5
  p = ''.join(x)
  if kdf(bytes(p, 'ascii')) == target:
    print(p)
    break

In [ ]:
%time
hashPasswordTable[target]

## Úkol 2
Pro zamezení nebo ztížení využití předpočítaných tabulek se využívá sůl.

Prostá hash hesla: hash = H (password)

Osolená hash: salted_hash = H (salt || password)

Uvažujme zjednodušený případ, kdy sůl může být pouze jeden malý znak (a-z).
Vytvořte novou předpočítanou tabulku, která bude obsahovat všechny možné kombinace soli a slov ze seznamu výše. (Pro naše účely se sůl jednoduše zřetězí se slovem před zahashováním, lze ji také zadat jako argument volání funkce `kdf`)

-------------------

Jak bude velká tato tabulka?

**Odpověď:**
Celková velikost: 318,971×26(sůl)×64 = 532,595,456 bytů, což je přibližně 508 MB.


Jak dlouhá by měla být sůl v případě, že bychom chtěli, aby výsledná předpočítaná tabulka byla větší než 1TB? Předpokládejte stále stejný slovník, do velikosti tabulky pro jednoduchost stačí uvažovat pouze velikost 1 hash a její počet (nemusíte zakomponovávat velikost řetězců reprezentující heslo a sůl).
U délky soli uvažujte 2 případy:
1) Sůl sestávající z malých znaků (a-z)
2) Sůl sestávající z libovolných bytů (hexadecimálně 0x00-0xFF)
(Může být výhodné si nejprve vyjádřit, jakou entropii by sůl měla mít.)

**Odpověď:** 1,099,511,627,776 bytů / (64 bytů x 318,971 ) = 5,386 (komb.)
1)Sůl o délce 3: 26^3=17,576 (komb. > 5,386 ) 2) Sůl o délce 2: 256^2 =65,536 (komb. > 5,386 )

-------------------
Poté si zvolte náhodně sůl (1 znak) a 1 slovo ze seznamu, které poslouží jako vaše heslo. Tuto kombinaci zahashujte, vzájemně si pošlete ve dvojicích a zjistěte heslo vašeho kolegy.

**Poznámka:** Kromě samotné hashe můžete kolegovi prozradit i sůl. V běžném scénáři (únik databáze) jsou k dispozici všechny údaje nutné pro výpočet hashe (použitý algoritmus, sůl, počet iterací), kromě samotného hesla.


In [15]:
saltedhashPasswordTable = kdf(bytes(passwords[2], 'utf8'))

Kromě soli se pro ztížení útoků využívá také vyšší počet iterací vybrané hashovací funkce.

In [12]:
%%time
kdf(b'abcdefgh', rounds=10000)

CPU times: total: 0 ns
Wall time: 7.97 ms


b'\xbb,;(!\x8eb\xc9\x9a \xaa\xdfS\x8b\xee\xe0\xcbsKR\x9aT\xfa\xd3d\x8c?\xf2\x81\xfd\xe9\x8e*\xfd[uG\x9dM\xb4>e\xaeP\xd6\x9f$\xad\xaf\xc2\xf3/ \xc8m\xbdG\xf7]\xa1\x08\xa4t\xc4'

In [17]:
start_time = time.time()
number_of_hashes = 1000
for _ in range(number_of_hashes):
    kdf(b'test')
end_time = time.time()
hashes_per_second = number_of_hashes / (end_time - start_time)

print(f"Hash rate: {hashes_per_second} hashes/second")

Hash rate: 1016308.2141991762 hashes/second


## Úkol 3

Spočtěte váš hash rate (počet hashů za vteřinu, které dokážete spočítat).

**Odpověď:**
1016308.2141991762 hashes/second

Kolik iterací hashovací fce bude potřeba nastavit při tvorbě hashe z hesla, aby útočníkovi trvalo přibližně rok jeho prolomení? Předpokládejme, že heslo je vytvořeno zcela náhodně z tisknutelných ASCII znaků (95), je dlouhé 8 znaků, hash rate útočníka je 1000000000000 hash/vteřina (1 terahash/s). Jak dlouho by v takovém případě trval výpočet hash z hesla na vašem zařízení?

**Odpověď:**
1) 95^8 komb. hesel / 1000000000000 hash/vteřina = 6634.20431 second (prolomení). 31,536,000 second (v roce) / 95^8 komb. hesel = 4752 iteraci
2) Moje zařízení: 1 / 1016308.2141991762 hashes/second = 0.000000984 sekund. 4752 x 0.000000984 = 0.004678 sekund
 
